# 06 — Macro Dashboard (Streamlit)

**Purpose**: Live monitoring dashboard — NOT discovery. This is compressed wisdom.

**Panels**:
1. Current Regime
2. Sector Sensitivities
3. Macro Shock Monitor
4. Global Spillovers

**Run with**: `streamlit run dashboard.py`

---

This notebook generates the `dashboard.py` file for deployment.

In [1]:
import pandas as pd
from pathlib import Path

PROCESSED_PATH = Path('../data_processed')
NOTEBOOK_PATH = Path('.')

print("Generating Streamlit dashboard...")

Generating Streamlit dashboard...


In [2]:
dashboard_code = r'''
"""
India Macro Research Dashboard
==============================
Institutional-grade macro monitoring system.

Run: streamlit run dashboard.py
"""

import streamlit as st
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

st.set_page_config(
    page_title="India Macro Dashboard",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded"
)

DATA_PATH = Path("data_processed")

@st.cache_data(ttl=3600)
def load_data():
    data = {}
    files = ["macro_regime_states", "sector_regime_performance", "macro_sector_correlation", "stress_test_impacts", "rule_signals"]
    for file in files:
        path = DATA_PATH / f"{file}.parquet"
        if path.exists():
            data[file] = pd.read_parquet(path)
    return data

try:
    data = load_data()
    data_loaded = True
except Exception as e:
    data_loaded = False
    st.error(f"Error loading data: {e}")

st.title("📊 India Macro Research Dashboard")
st.markdown("*Institutional-grade macro monitoring*")

if data_loaded:
    st.header("1️⃣ Current Macro Regime")
    if "macro_regime_states" in data:
        regime_df = data["macro_regime_states"]
        if "COMPOSITE" in regime_df.columns:
            current_regime = regime_df["COMPOSITE"].dropna().iloc[-1]
            regime_colors = {"Goldilocks": "#2ecc71", "Reflation": "#f39c12", "Stagflation": "#e74c3c", "Deflation": "#3498db"}
            col1, col2, col3 = st.columns(3)
            with col1:
                color = regime_colors.get(current_regime, "#95a5a6")
                st.markdown(f"<div style='background-color:{color}; padding:20px; border-radius:10px; text-align:center;'><h2 style='color:white; margin:0;'>{current_regime}</h2><p style='color:white; margin:0;'>Current Regime</p></div>", unsafe_allow_html=True)
    
    st.divider()
    st.header("2️⃣ Sector Sensitivities")
    if "sector_regime_performance" in data:
        pivot = data["sector_regime_performance"].pivot(index="sector", columns="regime", values="annualized_return")
        st.plotly_chart(px.imshow(pivot, color_continuous_scale="RdYlGn", title="Sector Returns by Regime (%)"), use_container_width=True)
    
    st.divider()
    st.header("3️⃣ Active Macro Rules")
    if "rule_signals" in data:
        active = data["rule_signals"].iloc[-1]
        active = active[active == True].index.tolist()
        if active: st.success(f"🟢 Active rules: {', '.join(active)}")
        else: st.info("⚪ No rules currently active")
    
    st.divider()
    st.header("4️⃣ Stress Test Monitor")
    if "stress_test_impacts" in data:
        stress_df = data["stress_test_impacts"]
        scenario = st.selectbox("Select Scenario", stress_df.columns.tolist())
        impact = stress_df[scenario].sort_values()
        fig = go.Figure(go.Bar(y=impact.index, x=impact.values, orientation="h", marker_color=np.where(impact.values > 0, "green", "red")))
        st.plotly_chart(fig, use_container_width=True)
else:
    st.warning("Please run the analysis notebooks first to generate data.")
'''

dashboard_path = NOTEBOOK_PATH.parent / 'dashboard.py'
with open(dashboard_path, 'w', encoding='utf-8') as f:
    f.write(dashboard_code)

requirements = """pandas>=2.0.0
numpy>=1.24.0
matplotlib>=3.7.0
seaborn>=0.12.0
scipy>=1.11.0
yfinance>=0.2.30
pyarrow>=14.0.0
streamlit>=1.28.0
plotly>=5.18.0
"""
with open(NOTEBOOK_PATH.parent / 'requirements.txt', 'w', encoding='utf-8') as f:
    f.write(requirements)

print(f"✓ Dashboard and requirements saved with UTF-8 encoding")

✓ Dashboard and requirements saved with UTF-8 encoding
